In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
# En caso de que se ejecute clonando el repositorio

DATA_DIR = Path.cwd().resolve().parent / "datos"

datos_titanic = pd.read_parquet(DATA_DIR / "02_datos_con_tipo_de_dato_ajustado_titanic.parquet", engine="pyarrow")

## Selección de Columnas y Estructura del DataFrame

In [3]:
columnas_seleccionadas = [
    "pclass",
    "sex",
    "age",
    "sibsp",
    "parch",
    "fare",
    "embarked",
    "survived",
]

In [4]:
df_titanic = datos_titanic[columnas_seleccionadas]

df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   pclass    1309 non-null   int64   
 1   sex       1309 non-null   category
 2   age       1046 non-null   float64 
 3   sibsp     1309 non-null   int8    
 4   parch     1309 non-null   int8    
 5   fare      1308 non-null   float64 
 6   embarked  1307 non-null   category
 7   survived  1309 non-null   bool    
dtypes: bool(1), category(2), float64(2), int64(1), int8(2)
memory usage: 37.5 KB


# AutoML


A continuación, se muestra un ejemplo **completamente documentado** sobre cómo usar [PyCaret](https://pycaret.gitbook.io/docs#classification) para llevar a cabo un proceso de **preprocesamiento y selección de modelos** de manera similar a lo que hace un flujo de trabajo con *ColumnTransformer* y *Pipeline* en scikit-learn.

## ¿Por Qué No Se Usa Pipeline ni ColumnTransformer Directamente?

PyCaret **automatiza** todo el proceso de:
- Imputación de valores nulos  
- Codificación de variables categóricas (One-Hot o Label Encoding)  
- Opciones de escalado/normalización (si lo solicitas)  
- Selección y entrenamiento de múltiples algoritmos  

Internamente, `PyCaret` construye su propio pipeline con estos pasos – no tienes que definirlo manualmente. Por esta razón, **no** se ven explícitos `ColumnTransformer` ni `Pipeline`, como lo harías en código puro de scikit-learn. `PyCaret`, al llamar a la función `setup`, crea un pipeline que incluye las transformaciones de preprocesamiento y lo aplica automáticamente a todos los modelos que compara o entrena.

In [5]:
# Importar las funciones principales de clasificación en PyCaret

from pycaret.classification import setup, compare_models

In [6]:
# Definir la columna objetivo
target_col = "survived"

Para leer más de la documentación oficial del método `setup` dirigirse a:

https://pycaret.readthedocs.io/en/stable/api/classification.html

In [8]:
# Llamada a setup
clf_setup = setup(
    data=df_titanic,          # DataFrame con TODAS las columnas
    target=target_col,        # Nombre de la columna objetivo
    session_id=42,            # Semilla para reproducibilidad (opcional pero recomendable)

    # Parámetros para controlar el preprocesamiento:
    numeric_features=["age", "fare", "sibsp", "parch"],  # Columnas que consideramos numéricas
    categorical_features=["sex", "embarked"],            # Columnas categóricas sin orden
    ordinal_features={"pclass": [1, 2, 3]},              # pclass es una variable categórica con orden

    # Estrategias de imputación
    numeric_imputation="median",  # Reemplaza valores nulos numéricos con la mediana
    categorical_imputation="mode", # Reemplaza valores nulos categóricos con el valor más frecuente

    # Con esto PyCaret creará automáticamente un pipeline que hace:
    # 1) Imputación de NaN en columnas numéricas con 'median'
    # 2) Imputación de NaN en columnas categóricas con 'mode'
    # 3) Encoding ordinal para pclass
    # 4) Encoding one-hot o label para otras columnas categóricas

)

,Description,Value
0,Session id,42
1,Target,survived
2,Target type,Binary
3,Original data shape,"(1309, 8)"
4,Transformed data shape,"(1309, 10)"
5,Transformed train set shape,"(916, 10)"
6,Transformed test set shape,"(393, 10)"
7,Ordinal features,1
8,Numeric features,4
9,Categorical features,2


En `PyCaret`, no necesitas separar manualmente los datos en conjuntos de entrenamiento y prueba. Esto se hace automáticamente al llamar a la función `setup`. A continuación, se explica el proceso y cómo ajustar (fine-tune) el modelo GBC.

## Comparación de varios modelos


La forma más sencilla de probar múltiples algoritmos de clasificación es llamar a compare_models. Por defecto, probará un conjunto amplio (más de 10) de algoritmos. Podemos restringirlo con el parámetro `include`.

In [9]:
best_model = compare_models(
    include=["lr", "rf", "gbc", "lightgbm"],  # Solo estos 4 algoritmos
    sort="Accuracy",   # Métrica principal para comparar
    n_select=4         # Selecciona y devuelve los 4 mejores
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7957,0.8448,0.6629,0.7703,0.7104,0.5547,0.5598,0.0110
lightgbm,Light Gradient Boosting Machine,0.7946,0.8342,0.7086,0.7450,0.7243,0.5612,0.5635,0.5290
rf,Random Forest Classifier,0.7836,0.8358,0.6943,0.7291,0.7088,0.5373,0.5400,0.1540
lr,Logistic Regression,0.7739,0.8235,0.6686,0.7209,0.6906,0.5137,0.5171,0.2810


En una inspección rápida, el mejor modelo fue el **Gradient Boosting Classifier**, con la mayor métrica en `Accuracy`. Por lo tanto, exploremos este modelo un poco más y ajsutémoslo.

In [14]:
from pycaret.classification import create_model, tune_model, predict_model

# 1. Crear el modelo GBC por defecto
print("Creando modelo GBC...")
gbc_model = create_model("gbc")

print("\n")
print("--------------------------------------------------")
print("\n")

# 2. Afinar hiperparámetros
print("Realizando fine-tuning del modelo GBC...")
gbc_tuned = tune_model(gbc_model)

print("\n")
print("--------------------------------------------------")
print("\n")

# 3. Mostrar el resumen del modelo ajustado
print("Resumen del modelo ajustado:")
print(gbc_tuned)  # Esto imprimirá un resumen con los hiperparámetros y rendimiento

print("\n")
print("--------------------------------------------------")
print("\n")

# 4. Predecir sobre la partición interna de PyCaret
print("Generando predicciones en el conjunto de validación interno:")
predicciones_internas = predict_model(gbc_tuned)

print("\n")
print("--------------------------------------------------")
print("\n")

print(predicciones_internas.head())

Creando modelo GBC...


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7826,0.7997,0.6286,0.7586,0.6875,0.5231,0.5285
1,0.8043,0.8692,0.7143,0.7576,0.7353,0.5803,0.5810
2,0.7935,0.8657,0.6857,0.7500,0.7164,0.5545,0.5559
3,0.8587,0.9298,0.7714,0.8438,0.8060,0.6952,0.6969
4,0.8804,0.9461,0.7714,0.9000,0.8308,0.7392,0.7444
5,0.8152,0.8612,0.7143,0.7812,0.7463,0.6014,0.6029
6,0.7802,0.8408,0.7143,0.7143,0.7143,0.5357,0.5357
7,0.7253,0.7964,0.4857,0.7083,0.5763,0.3833,0.3982
8,0.7363,0.7571,0.5714,0.6897,0.6250,0.4244,0.4288




--------------------------------------------------


Realizando fine-tuning del modelo GBC...


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7500,0.8093,0.6286,0.6875,0.6567,0.4608,0.4619
1,0.7935,0.8842,0.7143,0.7353,0.7246,0.5595,0.5596
2,0.8152,0.8712,0.7714,0.7500,0.7606,0.6102,0.6103
3,0.8696,0.9373,0.8000,0.8485,0.8235,0.7202,0.7210
4,0.8261,0.9060,0.7429,0.7879,0.7647,0.6270,0.6276
5,0.7935,0.8637,0.6857,0.7500,0.7164,0.5545,0.5559
6,0.7692,0.8242,0.6571,0.7188,0.6866,0.5045,0.5058
7,0.7473,0.7980,0.5143,0.7500,0.6102,0.4326,0.4495
8,0.7692,0.7689,0.6571,0.7188,0.6866,0.5045,0.5058


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


--------------------------------------------------


Resumen del modelo ajustado:
GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


--------------------------------------------------


Generando predicciones en el

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8193,0.8825,0.7200,0.7883,0.7526,0.6108,0.6123




--------------------------------------------------


      pclass     sex   age  sibsp  parch       fare embarked  survived  \
1289       3    male  21.0      1      0   6.495800        S     False   
352        2  female  40.0      1      1  39.000000        S      True   
1189       3  female   4.0      1      1  16.700001        S      True   
78         1  female  64.0      0      2  83.158302        C      True   
974        3    male  30.0      1      0  16.100000        S     False   

      prediction_label  prediction_score  
1289                 0            0.9167  
352                  1            0.9234  
1189                 1            0.7879  
78                   1            0.9369  
974                  0            0.8676  


## Entendiendo la Columna "prediction_score"

La columna **prediction_score** no indica que la predicción sea 100% o 0%; en cambio, representa la **confianza** o **probabilidad** asignada por el modelo a que una muestra pertenezca a una determinada clase.

### ¿Qué es "prediction_score"?

- **Probabilidad de Clase:**  
  La mayoría de los modelos de clasificación generan una probabilidad para cada clase (por ejemplo, la probabilidad de que la muestra pertenezca a la clase 1).  

  - Por ejemplo, un score de 0.7050 significa que el modelo cree que hay un 70.5% de probabilidad de que la muestra sea de la clase 1.
  
- **Umbral de Decisión:**  

  Se establece un umbral (típicamente 0.5) para convertir la probabilidad en una predicción binaria.  

  - Si el score es mayor que 0.5, la predicción se etiqueta como 1.  
  - Si es menor, se etiqueta como 0.
  
- **Calibración y Modelado:**  

  La puntuación rara vez es exactamente 1 o 0 porque los modelos operan con incertidumbre inherente.

  - Un score cercano a 1 indica alta confianza, pero es normal que incluso las predicciones correctas tengan un valor menor a 1.

  - Esto permite evaluar qué tan "seguro" está el modelo de cada predicción, lo que puede ser útil para decisiones posteriores.


### Conclusión

- La **prediction_score** es una medida de **confianza** y no un indicador de corrección absoluta.
- Las predicciones se obtienen al aplicar un umbral (usualmente 0.5) a estos scores.
- Es normal que incluso las predicciones correctas no tengan un score de 1, ya que el modelo siempre maneja un grado de incertidumbre en sus predicciones.


## ¿Cómo obtengo los parámetros del mejor modelo?

Una vez que has ajustado (fine-tuned) el modelo en PyCaret, el objeto devuelto (por ejemplo, `gbc_tuned`) es un estimador de scikit-learn. Para ver su configuración (hiperparámetros) puedes usar el método `get_params()`. Por ejemplo:

In [15]:
# Imprimir los parámetros del mejor modelo ajustado
print(gbc_tuned.get_params())

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


## Probar el modelo en datos no vistos

In [16]:
# Número de muestras sintéticas
np.random.seed(57)

n_samples = 50

# Generar datos sintéticos siguiendo la estructura del dataset Titanic
df_synthetic_test = pd.DataFrame({
    "pclass": np.random.choice([1, 2, 3], size=n_samples),
    "sex": np.random.choice(["male", "female"], size=n_samples),
    "age": np.random.uniform(0, 80, size=n_samples),
    "sibsp": np.random.randint(0, 4, size=n_samples),
    "parch": np.random.randint(0, 4, size=n_samples),
    "fare": np.random.uniform(10, 100, size=n_samples),
    "embarked": np.random.choice(["C", "Q", "S"], size=n_samples)
})

# Mostrar los primeros registros para verificar
df_synthetic_test.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,29.549676,2,2,87.460519,C
1,2,male,22.782733,2,1,81.690288,S
2,3,male,74.169372,1,2,51.914214,Q
3,1,male,62.041734,3,2,74.020331,Q
4,3,male,51.305236,0,0,26.458023,S


### 💥❗ Garantizar que a los Datos Sintéticos se les Aplique el Mismo Preprocesamiento :


Cuando configuraste `PyCaret` con la función `setup`, se creó internamente un pipeline que realiza:

* Imputación de valores faltantes (según las estrategias definidas).

* Codificación de variables numéricas, categóricas y ordinales.


Este pipeline se guarda y se aplica automáticamente a cualquier dato nuevo que pases a la función predict_model.

In [18]:
# Aplicar el modelo ajustado a los datos sintéticos de prueba
predicciones_sinteticas = predict_model(gbc_tuned, data=df_synthetic_test)

# Mostrar las predicciones (incluye 'prediction_label' y 'prediction_score')
predicciones_sinteticas.head()

,pclass,sex,age,sibsp,parch,fare,embarked,prediction_label,prediction_score
0,3,male,29.549675,2,2,87.460518,C,0,0.6548
1,2,male,22.782732,2,1,81.690285,S,0,0.8137
2,3,male,74.169373,1,2,51.914215,Q,0,0.9163
3,1,male,62.041733,3,2,74.020332,Q,0,0.8994
4,3,male,51.305237,0,0,26.458023,S,0,0.7674


# Guardar el Modelo

In [19]:
from pycaret.classification import save_model

In [ ]:
# Subir un nivel y crear una carpeta 'modelos' si no existe

MODELS_DIR = Path.cwd() / "modelos"
MODELS_DIR.mkdir(exist_ok=True)

# Guardar el modelo en un archivo llamado 'gbc_tuned_model.pkl'

model_path = MODELS_DIR / 'gbc_tuned_model'
save_model(gbc_tuned, model_name=str(model_path))

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['age', 'fare', 'sibsp', 'parch'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='median'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None, include=['sex', 'emba...
                                             criterion='friedman_mse', init=None,
                                             learning_rate=0.1, loss='log_loss',
              

# Cargar el modelo

In [27]:
from pycaret.classification import load_model

# Cargar el modelo previamente guardado
gbc_tuned_loaded = load_model(str(MODELS_DIR /'gbc_tuned_model'))

Transformation Pipeline and Model Successfully Loaded


# Exportar en otros formatos

Si prefieres utilizar joblib para guardar el modelo (ya que también es común y a veces más rápido para objetos grandes), puedes hacerlo manualmente, ya que el modelo devuelto por PyCaret es compatible con scikit-learn. Por ejemplo:

In [29]:
import joblib


# Exportar el modelo usando joblib en lugar de pickle
joblib.dump(gbc_tuned, MODELS_DIR / 'gbc_tuned_model.joblib')

# Para cargarlo luego:
gbc_tuned_loaded = joblib.load(MODELS_DIR / 'gbc_tuned_model.joblib')